In [ ]:
!sudo apt update
!sudo apt-get install openjdk-21-jre
!wget https://dlcdn.apache.org/spark/spark-3.5.5/spark-3.5.5-bin-hadoop3.tgz
!tar -xf spark-3.5.5-bin-hadoop3.tgz
!pip install -q findspark

In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-1.21.0-openjdk-amd64"
os.environ["SPARK_HOME"] = "./spark-3.5.5-bin-hadoop3"

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.master("local[*]").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/22 16:04:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
recetas = spark.read.option("inferSchema","true").option("header","false").csv("recetas.csv")
#recetas.printSchema()

header = spark.read.option("inferSchema","true").option("header","true").csv("header.csv")
header_c = header.union(recetas)
header_c2 = header_c.filter(header_c["any"] != "any")

recetas2 = header_c2.select(header_c2["any"].alias("any"), header_c2["regió sanitària"].alias("rsanitaria"), header_c2["sexe"].alias("sexe"), header_c2["grup ATC nivell 4"].alias("medicament"), header_c2["nombre de receptes"].alias("nreceptes"), header_c2["import íntegre"].alias("import"))
recetas2.show(10)

df = header_c2

+----+----------+----+--------------------+---------+-------+
| any|rsanitaria|sexe|          medicament|nreceptes| import|
+----+----------+----+--------------------+---------+-------+
|2022|    LLEIDA|Dona|Antiinfecciosos y...|        8|  22.64|
|2022|    LLEIDA|Dona|Inhibidores de la...|        2|  34.68|
|2022|    LLEIDA|Dona|        Antibioticos|        1|   3.59|
|2022|    LLEIDA|Dona|Formulaciones de ...|       36| 115.08|
|2022|    LLEIDA|Dona|Vitamina D y anal...|      146|1461.00|
|2022|    LLEIDA|Dona|    Vitamina B1 sola|        3|  46.56|
|2022|    LLEIDA|Dona|Otros preparados ...|       14|  91.32|
|2022|    LLEIDA|Dona|Grupo de la heparina|       15|  60.90|
|2022|    LLEIDA|Dona|Acido folico y de...|        1|   2.50|
|2022|    LLEIDA|Dona|Soluciones que af...|       14|  30.13|
+----+----------+----+--------------------+---------+-------+
only showing top 10 rows



Q1-How many drugs were prescribed during 2022, how many prescriptions and which was the overall cost?

In [4]:
recetas2.createOrReplaceTempView("recetas2")

consulta_q1 = """
    SELECT
    COUNT(DISTINCT medicament) AS num_medicaments,
    SUM(CAST(nreceptes AS INT)) AS total_receptes,
    SUM(CAST(import AS DOUBLE)) AS cost_total
    FROM recetas2
    WHERE any = 2022
"""
spark.sql(consulta_q1).show()

+---------------+--------------+-------------------+
|num_medicaments|total_receptes|         cost_total|
+---------------+--------------+-------------------+
|            367|      23588623|3.125160864800002E8|
+---------------+--------------+-------------------+



Q2-Which is the most prescribed drug in men and women and in which sanitary region?

In [6]:
consulta_q2 = """
SELECT sexe, rsanitaria, medicament, SUM(CAST(nreceptes AS INT)) AS total_receptes
FROM recetas2
GROUP BY sexe, rsanitaria, medicament
ORDER BY sexe, total_receptes DESC
"""
spark.sql(consulta_q2).show()

+------+----------+--------------------+--------------+
|  sexe|rsanitaria|          medicament|total_receptes|
+------+----------+--------------------+--------------+
|Altres|    ALTRES|   AGONISTES OPIACIS|          7546|
|Altres|    ALTRES|   Sense especificar|          3506|
|Altres|    ALTRES|Derivados de la b...|           134|
|Altres|    ALTRES|Inhibidores de la...|            68|
|Altres|    ALTRES|Inhibidores de la...|            58|
|Altres|    ALTRES|            Anilidas|            53|
|Altres|    ALTRES|Inhibidores de la...|            50|
|Altres|    ALTRES|Inhibidores selec...|            47|
|Altres|    ALTRES|Vitamina D y anal...|            45|
|Altres|    ALTRES|          Biguanidas|            45|
|Altres|    ALTRES|Agentes beta- blo...|            41|
|Altres|    ALTRES|Inhibidores de la...|            37|
|Altres|    ALTRES|Otros antihistami...|            29|
|Altres|    ALTRES|Derivados del aci...|            27|
|Altres|    ALTRES|Sulfonamidas, mon...|        

Q3-Which is the least prescribed drug in men and women and in which sanitary region?

In [7]:
consulta_q3d = """
SELECT sexe, rsanitaria, medicament, SUM(CAST(nreceptes AS INT)) AS total_receptes
FROM recetas2
WHERE sexe = 'Dona'
GROUP BY sexe, rsanitaria, medicament
ORDER BY total_receptes ASC
LIMIT 1
"""
spark.sql(consulta_q3d).show()

+----+-----------------+--------------------+--------------+
|sexe|       rsanitaria|          medicament|total_receptes|
+----+-----------------+--------------------+--------------+
|Dona|SENSE ESPECIFICAR|Derivados del tio...|             1|
+----+-----------------+--------------------+--------------+



In [11]:
consulta_q3h = """
SELECT sexe, rsanitaria, medicament, SUM(CAST(nreceptes AS INT)) AS total_receptes
FROM recetas2
WHERE sexe = 'Home'
GROUP BY sexe, rsanitaria, medicament
ORDER BY total_receptes ASC
LIMIT 1
"""
spark.sql(consulta_q3h).show()

+----+----------------+--------------------+--------------+
|sexe|      rsanitaria|          medicament|total_receptes|
+----+----------------+--------------------+--------------+
|Home|TERRES DE L'EBRE|Derivados de la q...|             1|
+----+----------------+--------------------+--------------+



Q4-Which is the most expensive drug prescribed?

In [8]:
consulta_q4 = """
SELECT medicament,
       SUM(CAST(import AS FLOAT)) AS total_import,
       SUM(CAST(nreceptes AS INT)) AS total_receptes,
       (SUM(CAST(import AS FLOAT)) / SUM(CAST(nreceptes AS INT))) AS preu_mig_per_recepta
FROM recetas2
GROUP BY medicament
ORDER BY preu_mig_per_recepta DESC
LIMIT 1
"""
spark.sql(consulta_q4).show()


+--------------------+--------------------+--------------+--------------------+
|          medicament|        total_import|total_receptes|preu_mig_per_recepta|
+--------------------+--------------------+--------------+--------------------+
|Otras hormonas de...|1.1603210341308594E7|          3213|  3611.3321946182987|
+--------------------+--------------------+--------------+--------------------+



Q5-Considering the top 10 of most prescribed drugs during 2022, show the cost of the drugs for each sanitary region

In [12]:
top10_query = """
SELECT medicament
FROM recetas2
WHERE any = 2022
GROUP BY medicament
ORDER BY SUM(CAST(nreceptes AS INT)) DESC
LIMIT 10
"""
spark.sql(top10_query).createOrReplaceTempView("top10_meds")

spark.sql("""
SELECT rsanitaria, medicament, SUM(CAST(import AS DOUBLE)) AS cost_total
FROM recetas2
WHERE any = 2022 AND medicament IN (SELECT medicament FROM top10_meds)
GROUP BY rsanitaria, medicament
ORDER BY rsanitaria, cost_total DESC
""").show(100)


+------------------+--------------------+--------------------+
|        rsanitaria|          medicament|          cost_total|
+------------------+--------------------+--------------------+
|ALT PIRINEU i ARAN|   Sense especificar|  204690.41000000006|
|ALT PIRINEU i ARAN|Inhibidores de la...|   76283.83000000002|
|ALT PIRINEU i ARAN|Inhibidores de la...|  60614.150000000016|
|ALT PIRINEU i ARAN|Inhibidores selec...|            41702.11|
|ALT PIRINEU i ARAN|Inhibidores de la...|   33953.54000000001|
|ALT PIRINEU i ARAN|Derivados de la b...|  29643.290000000005|
|ALT PIRINEU i ARAN|            Anilidas|  21570.919999999995|
|ALT PIRINEU i ARAN|Inhibidores de la...|  19464.149999999998|
|ALT PIRINEU i ARAN|Derivados del aci...|  17350.679999999997|
|ALT PIRINEU i ARAN|Agentes beta- blo...|  13126.879999999996|
|            ALTRES|Inhibidores de la...|               56.55|
|            ALTRES|Inhibidores selec...|               31.35|
|            ALTRES|Derivados de la b...|              